In [ ]:

import numpy as np
import pandas as pd 


from transformers import AutoTokenizer, AutoModelForMaskedLM


from langchain.embeddings.huggingface import HuggingFaceEmbeddings


df=pd.read_csv("./all_poems.csv")


data=df['poem_text']

data.head()


df.head()

In [ ]:

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
embedder = HuggingFaceEmbeddings(model_name="asafaya/bert-base-arabic")




In [ ]:
data_list=data.tolist()
print(len(data_list))

In [ ]:
import random

random_indices=[random.randint(0,58021) for i in range(100)]


data_list=data.iloc[random_indices].to_list()
test_emeb=embedder.embed_documents(data_list)


In [ ]:
print(len(test_emeb))
print(len(data[0]))
print(len(test_emeb[0]))

print(data[0][0])

# a code to insert into a postgress db


In [ ]:
import psycopg2

# Define your database credentials
db_name = "hawiya"
user = "postgres"
password = "password"
host = "localhost"
port = "5432"

# Establish a connection to the database
conn = psycopg2.connect(database=db_name, user=user, password=password, host=host, port=port)
cur = conn.cursor()
#cur.execute("CREATE TABLE embeddings (id bigserial PRIMARY KEY, word TEXT, embedding vector(" + str(768)+ "));")
conn.commit()



# Iterate over the word embeddings
for  i,emebeding in enumerate(test_emeb):
    # Convert the embedding to a string representation
    
    my_doc = {"id":i, "text" :data[i],"embedding":emebeding}
    
    # Insert the embedding into the database
    cur.execute("""INSERT INTO embeddings (id,word,embedding) VALUES (%(id)s, %(text)s, %(embedding)s)""", my_doc)
    i=i+1
    

# Commit the changes and close the connection
conn.commit()
conn.close()


A smaple code that do similarity search using cosine


In [ ]:
from sklearn.metrics.pairwise import cosine_distances
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
# Assuming your query is a string stored in the variable 'query'
query="المغركة"

# Calculate the embedding for the query
query_embedding = await embedder.aembed_documents([query])



# Calculate cosine similarities between the query and all documents
similarities = cosine_distances(query_embedding, test_emeb)
print(similarities)

# Get the index of the most similar document
most_similar_index = similarities.argmax()

# Print the most similar document
print(data_list[most_similar_index])


# use another emebderrer and re run the cells above


In [ ]:
embedder = HuggingFaceEmbeddings(model_name="elgeish/gpt2-medium-arabic-poetry")

# Here I used a word emebedding model


In [45]:


import gensim
import re
import numpy as np
from nltk import ngrams
from utilities import * 

model = gensim.models.Word2Vec.load('./models/full_grams_cbow_100_wiki.mdl')


token = clean_str(u'دين').replace(" ", "_")


if token in model.wv:
    most_similar = model.wv.most_similar( token, topn=10 )
    for term, score in most_similar:
        term = clean_str(term).replace(" ", "_")
        if term != token:
            print(term, score)



pos_tokens=[ clean_str(t.strip()).replace(" ", "_") for t in [] if t.strip() != ""]
neg_tokens=[ clean_str(t.strip()).replace(" ", "_") for t in [''] if t.strip() != ""]

vec = calc_vec(pos_tokens=pos_tokens, neg_tokens=neg_tokens, n_model=model, dim=model.vector_size)

most_sims = model.wv.similar_by_vector(vec, topn=10)
for term, score in most_sims:
    if term not in pos_tokens+neg_tokens:
        print(term, score)


word_vector = model.wv[ token ]

word_vector


معتقد 0.6983626484870911
ديانه 0.6978577971458435
مذهب 0.6649177074432373
ودين 0.6610172986984253
الدين 0.6588460206985474
دين_الاسلام 0.6555569767951965
بدين 0.6483359336853027
ملحد 0.6435856819152832
ديانته 0.6386861801147461
الاديان 0.6349233388900757
الابزمو 0.0
الدائم_الجريان 0.0
وقطيعه 0.0
مرحله_اخيره 0.0
جندو 0.0
السعره 0.0
تل_العبر 0.0
القملق 0.0
واشوريه 0.0
بالسرعه_القصوي 0.0


array([ 2.4409056 ,  3.9559472 ,  0.21397716, -0.521136  ,  0.36912638,
       -3.4426553 ,  0.28245908,  0.6631324 ,  1.621887  ,  0.9472866 ,
        0.31828615,  0.8806815 ,  2.157055  , -2.8736787 ,  2.787323  ,
       -1.1993876 , -0.93114215, -2.9399326 , -2.1674523 , -0.296613  ,
        2.231288  , -4.4799953 ,  2.0459082 , -2.9747512 , -0.15462679,
        1.4269712 ,  1.4808508 ,  2.2602508 , -4.9776835 ,  4.579959  ,
        1.8137403 ,  1.7823577 , -0.4286507 , -0.1906432 , -0.28240505,
       -1.5433767 , -0.8022959 , -1.2146631 ,  0.75317276, -1.761817  ,
        1.392355  ,  3.3090816 ,  1.9117038 , -3.9833324 ,  0.3098168 ,
       -0.75069016, -3.1657631 , -1.3959129 ,  1.3551639 ,  0.9554129 ,
        3.3537982 ,  4.023201  ,  0.82530016, -3.415946  , -2.3655834 ,
        0.04621622,  2.1432161 ,  1.1844045 , -2.884357  , -0.7491137 ,
        0.73062736, -0.17919174,  0.9194095 , -3.5709348 ,  0.6004428 ,
        2.945328  ,  0.70685315,  0.5640709 ,  2.7145057 , -3.23